# W_GL_BALANCE_F ETL Process
### Staging to Fact - Dimension Lookups and Incremental MERGE
**ODI Package:** SILOS_SIL_GLBALANCEFACT  
**IKM:** BIAPPS Oracle Fact Incremental Update

In [ ]:
%sql
-- Step 1: Variable IS_INCREMENTAL - Check if incremental load is enabled
CREATE OR REPLACE TEMP VIEW VAR_IS_INCREMENTAL AS
SELECT 
  CASE 
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL
FROM PRXBI_DW.W_ETL_LOAD_DATES
WHERE PACKAGE_NAME = 'SILOS_SIL_GLBALANCEFACT'
  AND (DATASOURCE_NUM_ID = 380 OR DATASOURCE_NUM_ID = 380)
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1'

In [ ]:
%sql
-- Step 2: Create Flow Table I$_2625555_1 with dimension lookups
DROP VIEW IF EXISTS I$_2625555_1;

CREATE OR REPLACE TEMP VIEW I$_2625555_1 AS
SELECT
  COALESCE(LKP_GL_ACCOUNT.ROW_WID, 0) AS GL_ACCOUNT_WID,
  COALESCE(LKP_PERIOD.ROW_WID, 0) AS PERIOD_WID,
  COALESCE(LKP_LEDGER.ROW_WID, 0) AS LEDGER_WID,
  FS.BALANCE_DATE,
  FS.DB_CR_IND,
  FS.BALANCE_ACCT_AMT,
  COALESCE(FS.BALANCE_LOC_AMT, 0) AS BALANCE_LOC_AMT,
  FS.ACTIVITY_ACCT_AMT,
  COALESCE(FS.ACTIVITY_LOC_AMT, 0) AS ACTIVITY_LOC_AMT,
  FS.ACCT_CURR_CODE,
  FS.LOC_CURR_CODE,
  FS.ADJUSTMENT_FLAG,
  FS.BALANCE_TYPE_FLAG,
  FS.TRANSLATED_FLAG,
  FS.CHANGED_ON_DT,
  FS.AUX1_CHANGED_ON_DT,
  FS.AUX2_CHANGED_ON_DT,
  FS.DATASOURCE_NUM_ID,
  FS.INTEGRATION_ID,
  FS.TENANT_ID,
  FS.X_CUSTOM
FROM PRXBI_DW.W_GL_BALANCE_FS FS
LEFT OUTER JOIN PRXBI_DW.W_GL_ACCOUNT_D LKP_GL_ACCOUNT
  ON FS.GL_ACCOUNT_ID = LKP_GL_ACCOUNT.INTEGRATION_ID
  AND FS.DATASOURCE_NUM_ID = LKP_GL_ACCOUNT.DATASOURCE_NUM_ID
  AND FS.BALANCE_DATE >= LKP_GL_ACCOUNT.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_ACCOUNT.EFFECTIVE_TO_DT
LEFT OUTER JOIN PRXBI_DW.W_MCAL_PERIOD_D LKP_PERIOD
  ON FS.PERIOD_ID = LKP_PERIOD.INTEGRATION_ID
  AND FS.DATASOURCE_NUM_ID = LKP_PERIOD.DATASOURCE_NUM_ID
LEFT OUTER JOIN PRXBI_DW.W_LEDGER_D LKP_LEDGER
  ON FS.LEDGER_ID = LKP_LEDGER.INTEGRATION_ID
  AND FS.DATASOURCE_NUM_ID = LKP_LEDGER.DATASOURCE_NUM_ID

In [ ]:
%sql
-- Step 3: Delete matched rows (for incremental update mode)
-- Note: In ODI, this step deletes existing records that will be updated
DELETE FROM PRXBI_DW.W_GL_BALANCE_F
WHERE (DATASOURCE_NUM_ID, INTEGRATION_ID) IN (
  SELECT DATASOURCE_NUM_ID, INTEGRATION_ID
  FROM I$_2625555_1
)

In [ ]:
%sql
-- Step 4: Insert new and updated rows into W_GL_BALANCE_F
INSERT INTO PRXBI_DW.W_GL_BALANCE_F
(
  GL_ACCOUNT_WID,
  PERIOD_WID,
  LEDGER_WID,
  BALANCE_DATE,
  DB_CR_IND,
  BALANCE_ACCT_AMT,
  BALANCE_LOC_AMT,
  ACTIVITY_ACCT_AMT,
  ACTIVITY_LOC_AMT,
  ACCT_CURR_CODE,
  LOC_CURR_CODE,
  ADJUSTMENT_FLAG,
  BALANCE_TYPE_FLAG,
  TRANSLATED_FLAG,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  TENANT_ID,
  X_CUSTOM,
  ETL_PROC_WID
)
SELECT
  GL_ACCOUNT_WID,
  PERIOD_WID,
  LEDGER_WID,
  BALANCE_DATE,
  DB_CR_IND,
  BALANCE_ACCT_AMT,
  BALANCE_LOC_AMT,
  ACTIVITY_ACCT_AMT,
  ACTIVITY_LOC_AMT,
  ACCT_CURR_CODE,
  LOC_CURR_CODE,
  ADJUSTMENT_FLAG,
  BALANCE_TYPE_FLAG,
  TRANSLATED_FLAG,
  CAST(CHANGED_ON_DT AS TIMESTAMP) AS CHANGED_ON_DT,
  CAST(AUX1_CHANGED_ON_DT AS TIMESTAMP) AS AUX1_CHANGED_ON_DT,
  CAST(AUX2_CHANGED_ON_DT AS TIMESTAMP) AS AUX2_CHANGED_ON_DT,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  TENANT_ID,
  X_CUSTOM,
  2625555 AS ETL_PROC_WID
FROM I$_2625555_1

In [ ]:
%sql
-- Step 5: Update reference dates in W_ETL_LOAD_DATES
MERGE INTO PRXBI_DW.W_ETL_LOAD_DATES AS TGT
USING (
  SELECT
    380 AS DATASOURCE_NUM_ID,
    'SILOS_SIL_GLBALANCEFACT' AS PACKAGE_NAME,
    'W_GL_BALANCE_F' AS TARGET_TABLE_NAME,
    '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
    2625555 AS ETL_PROC_WID,
    12345 AS LOAD_PLAN_ID,
    date_add(current_timestamp(), -30) AS WIP_LOAD_START_DATE,
    current_timestamp() AS ETL_LOAD_DATE,
    (SELECT CASE WHEN IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM VAR_IS_INCREMENTAL) AS COMMITTED
) SRC
ON TGT.DATASOURCE_NUM_ID = SRC.DATASOURCE_NUM_ID
  AND TGT.PACKAGE_NAME = SRC.PACKAGE_NAME
  AND TGT.ETL_USAGE_CODE = SRC.ETL_USAGE_CODE
WHEN MATCHED THEN
  UPDATE SET
    TGT.TARGET_TABLE_NAME = SRC.TARGET_TABLE_NAME,
    TGT.ETL_PROC_WID = SRC.ETL_PROC_WID,
    TGT.LOAD_PLAN_ID = SRC.LOAD_PLAN_ID,
    TGT.WIP_LOAD_START_DATE = SRC.WIP_LOAD_START_DATE,
    TGT.ETL_LOAD_DATE = SRC.ETL_LOAD_DATE,
    TGT.COMMITTED = SRC.COMMITTED
WHEN NOT MATCHED THEN
  INSERT (
    DATASOURCE_NUM_ID,
    PACKAGE_NAME,
    TARGET_TABLE_NAME,
    ETL_USAGE_CODE,
    ETL_PROC_WID,
    LOAD_PLAN_ID,
    WIP_LOAD_START_DATE,
    ETL_LOAD_DATE,
    COMMITTED
  )
  VALUES (
    SRC.DATASOURCE_NUM_ID,
    SRC.PACKAGE_NAME,
    SRC.TARGET_TABLE_NAME,
    SRC.ETL_USAGE_CODE,
    SRC.ETL_PROC_WID,
    SRC.LOAD_PLAN_ID,
    SRC.WIP_LOAD_START_DATE,
    SRC.ETL_LOAD_DATE,
    SRC.COMMITTED
  )

In [ ]:
%sql
-- Step 6: Insert reference dates to History Table W_ETL_LOAD_DATES_LOG
INSERT INTO PRXBI_DW.W_ETL_LOAD_DATES_LOG
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  2625555 AS SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
FROM PRXBI_DW.W_ETL_LOAD_DATES
WHERE DATASOURCE_NUM_ID = 380
  AND PACKAGE_NAME = 'SILOS_SIL_GLBALANCEFACT'
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 7: Drop work table (temp view)
DROP VIEW IF EXISTS I$_2625555_1;
DROP VIEW IF EXISTS VAR_IS_INCREMENTAL